In [1]:
# Turn on multi-threading on your computer for faster calculation 
%env OMP_NUM_THREADS = 4

env: OMP_NUM_THREADS=4


In [3]:
pip install dmba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 43.7 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


**Steps 1 and 2: Install and load the necessary packages and libraries**

In [5]:
from pathlib import Path # to interact with file system.

import numpy as np # for working with arrays.
import pandas as pd # for working with data frames (tables).
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split # for data partition.
from sklearn.metrics import r2_score # to identify r_squared for regression model.
from sklearn.linear_model import LinearRegression # for linear regression model.
from pandas.plotting import scatter_matrix, parallel_coordinates
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import dmba
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


%matplotlib inline 
import matplotlib.pylab as plt 

no display found. Using non-interactive Agg backend


**Step 3: Load Data**

In [ ]:
sales_df = pd.read_csv('train.csv')

In [ ]:
sales_df.head()

**Step 4: Explore data**

In [ ]:
sales_df.describe()

In [ ]:
sales_df.columns

**Step 5: Preprocess the data**

In [ ]:
sales_df.Company = sales_df.Company.astype('category')
sales_df.Quarter = sales_df.Quarter.astype('category')

In [ ]:
sales_df = sales_df.rename(columns={'Bond rating': 'Bond_rating'})

In [ ]:
sales_df = sales_df.rename(columns={'Stock rating': 'Stock_rating'})

In [ ]:
sales_df.Bond_rating = sales_df.Bond_rating.astype('category')
sales_df.Stock_rating = sales_df.Stock_rating.astype('category')
sales_df.Industry = sales_df.Industry.astype('category')
sales_df.Region = sales_df.Region.astype('category')

In [ ]:
sales_df.loc[:, 'Sales'] = sales_df.groupby(['ID', 'Quarter'])['Sales'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [ ]:
sales_df = sales_df.drop_duplicates(subset=['ID', 'Company'])

In [ ]:
sales_df.describe()

In [ ]:
test_df= pd.read_csv('test.csv')

In [ ]:
test_df = test_df.rename(columns={'Bond rating': 'Bond_rating'})

In [ ]:
test_df = test_df.rename(columns={'Stock rating': 'Stock_rating'})

In [ ]:
test_df.Stock_rating = test_df.Stock_rating.astype('category')
test_df.Bond_rating = test_df.Bond_rating.astype('category')
test_df.Industry = test_df.Industry.astype('category')
test_df.Company = test_df.Company.astype('category')
test_df.Quarter = test_df.Quarter.astype('category')
test_df.Region = test_df.Region.astype('category')

In [ ]:
sales_df.info()

In [ ]:
pd.DataFrame(sales_df).isna().sum()

In [ ]:
sales_df.describe()

In [ ]:
sales_df.dropna(subset=['Sales'], inplace=True)

In [ ]:
#Impute missing values in IR with mean

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Fit the imputer to columns with numerical data
imputer.fit(sales_df.select_dtypes(include=['int64', 'float64']))

# Transform and replace missing values in numerical columns wieith the mean
sales_df[sales_df.select_dtypes(include=['int64', 'float64']).columns] = imputer.transform(sales_df.select_dtypes(include=['int64', 'float64']))

In [ ]:
sales_df.describe()

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Fit the imputer to columns with numerical data
imputer.fit(test_df.select_dtypes(include=['int64', 'float64']))

# Transform and replace missing values in numerical columns with the mean
test_df[test_df.select_dtypes(include=['int64', 'float64']).columns] = imputer.transform(test_df.select_dtypes(include=['int64', 'float64']))

In [ ]:
def CombineRareCategories(data, mincount, columns_to_process):
    for col in columns_to_process:
        if (type(data[col][0]) == str):
            for index, row in pd.DataFrame(data[col].value_counts()).iterrows():
                if (row[0] < mincount):
                    data[col].replace(index, 'Other_' + col, inplace=True)
                else:
                    None

# Specify the columns you want to process
columns_to_process = ['Bond_rating', 'Stock_rating', 'Region','Industry']  # Add your column names here

# Apply the function to your data with the specified columns and mincount
CombineRareCategories(sales_df, 30, columns_to_process)

# Check the result for the first 10 rows
print(sales_df.head(10))

In [ ]:
features_to_normalize = ['MarketshareChange', 'RevenueGrowth','QuickRatio','InventoryRatio']

from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer(method='yeo-johnson')
sales_df[features_to_normalize] = scaler.fit_transform(sales_df[features_to_normalize])
scaler = PowerTransformer(method='yeo-johnson')
test_df[features_to_normalize] = scaler.fit_transform(test_df[features_to_normalize])

In [ ]:
sales_df.columns

**Step 6: Apply Machine learning Algorithm**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
predictors = ['ID','Company','Quarter', 'QuickRatio','InventoryRatio','RevenueGrowth', 'MarketshareChange',  'Bond_rating', 'Stock_rating',
       'Region','Industry']
outcome = 'Sales'

# partition data
X = pd.get_dummies(sales_df[predictors], drop_first=True)
y = sales_df[outcome]
# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Instantiate Ridge regression model with regularization parameter alpha
ridge_reg = Ridge(alpha=0.1)

# Fit the model to the training data
ridge_reg.fit(X_train, y_train)

# Make predictions

y_train_pred = ridge_reg.predict(X_train)
mae_train = mean_absolute_error(y_train, y_train_pred)
print("Train MAE: %f" % mae_train)

y_pred = ridge_reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Valid MAE: %f" % mae)

In [ ]:
test_df = pd.get_dummies(test_df, prefix_sep='_', drop_first=True)
sales_df = pd.get_dummies(sales_df, prefix_sep='_', drop_first=True)
all_column_names2 = test_df.columns.to_list()
all_column_names = sales_df.columns.to_list()

new_data = test_df[all_column_names2]
print(new_data) 
drop_columns = ['ID', 'Quarter_Q2','Bond_rating_Other_Bond_rating', 'Quarter_Q3', 'Bond_rating_BBB',
                'Bond_rating_CCC','Region_North','Stock_rating_Other_Stock_rating',
                'Quarter_Q4', 'Quarter_Q5', 'Quarter_Q6', 'Quarter_Q7', 
                'Quarter_Q8','Stock_rating_Strong Sell']

# If necessary, adjust fit_intercept
ridge_reg.fit(X_train.drop(drop_columns, axis=1), y_train)  # Remove Q5 if not needed

predicted_sales = ridge_reg.predict(new_data.drop(columns=['ID','Bond_rating_B','Bond_rating_CCC','Bond_rating_BBB','Region_North'], axis=1))
df = pd.DataFrame({
    'ID': new_data['ID'],
    'Sales': predicted_sales
})

# Display the DataFrame
print(df.head())
df['ID'] = df['ID'].astype(int)
#df.to_csv('submission40.csv', index=False)